In [356]:
import numpy as np
import pprint
import math
from functools import reduce
import operator

sample = """
_______________________________________________________________________
_______________________________________2>$_____________________________
_5_6-$-&_________________________________v__________3>$________________
_|/____|_____3-&-4_____________________9>$>$>$________v________________
_*-2___2_________________________________^__________6>$<2______________
_|___________7-*-$_______________________5____________v________________
_$_____________|_|__________________________________7>$>+>$>$__________
_______________$-$_________3>$>$>$>$__________________v_____v__________
_____________________________^_^_^_^__________________$_____$__________
_____________________________4_2_______________________________________
___2-*_________________________________________________________________
_____v_________________________________________________________________
___1>*<3________________________________________3>$____________________
_____v____________________________________________v____________________
___3>*__________________________________________6>*<2__________________
_____|____________________________________________v____________________
_____$__________________________________________7>+____________________
__________________________________________________v____________________
__________________________________________________$____________________
_______________________________________________________________________
_______________________________________________________________________
_______________________________________________________________________
_______________________________________________________________________
_______________________________________________________________________
_______________________________________________________________________
"""

directions = {
    '|': [0, 1, True],
    '/': [-1, 1, True],
    '-': [1, 0, True],
    '<': [-1, 0, False],
    '>': [1, 0, False],
    '^': [0, -1, False],
    'v': [0, 1, False]
}

initial = {
    '*': 1,
    '+': 0,
    '&': 1,
    '$': 0,
#     '%': 1,
}

operators = {
    '*': operator.mul,
    '+': operator.add,
}

def int_(val):
    try:
        return int(val)
    except:
        return 0

class Operator:
    def __init__(self, symbol, position=(0, 0)):
        self.symbol = symbol
        self.inputs = []
#         self.outputs = []
        self.value = 0
        if self.symbol in initial:
            self.value = initial[self.symbol]
        self.position = position
        self.op = None
        if self.symbol in operators:
            self.op = operators[self.symbol]
        else:
            self.op = operators['+']
        
    def evaluate(self):
        input_values = []
        for i in self.inputs:
            if type(i) is Operator:
                input_values.append(i.value)
            elif type(i) is str and i.isnumeric():
                input_values.append(int(i))
        if input_values:
            self.value = reduce(self.op, input_values)
        else:
            self.value = 0
        return self.value
    
    def __str__(self):
        return str(self.value)
    
class Locus:
    def __init__(self, program):
        lines = [list(l) for l in program.split('\n')[1:-1]]
        self.ops = '+*$&'
        self.grid = np.array(lines)
    #     size = len(lines[0]), len(lines)
        size = self.grid.shape
    #     new = ['_' * size[0]] * size[1]
    #     new = np.full(lines.shape, '_', dtype=object)
    
    def is_op(self, f):
        return (type(f) is str and f in self.ops) or type(f) is Operator
    
    def execute(self, i=3):
        t = tuple
        for iteration in range(i):
            new = self.grid.copy().astype(object)
            
            for x, y in np.ndindex(self.grid.shape):
                pos = (x, y)
                c = new[pos]
#                 if c in ops and False:
#         #             new[x][y] = 0
#                     if c in initial:
#                         n = initial[c]
#                     else:
#                         n = 0
#                     new[x][y] = n
                if type(c) is str and c in self.ops:
                    new[pos] = Operator(c, position=pos)
                elif type(c) is Operator:
                    c.inputs = []

#             Loop through cells in the grid
            for x, y in np.ndindex(self.grid.shape):
#                 Track whether an operation should be performed on the cells linked by the operator
                valid = False
                w = np.array([x, y])
                c = self.grid[x][y]
#                 The current cell is in the list of symbols used to "move" data
                if c in directions:
                    d = directions[c][:2]
                    d = np.flip(d)
                    a = self.grid[t(w+d)]
                    b = self.grid[t(w-d)]
                    
#                     Pipe is bidirectional
                    if directions[c][-1]:
#                         Determine which cell is the source and which is the destination
                        if self.is_op(a):
                            src = t(w-d)
                            dest = t(w+d)
                            valid = True
                        elif self.is_op(b):
                            src = t(w+d)
                            dest = t(w-d)
                            valid = True
#                     Pipe is not bidirectional
                    else:
                        if self.is_op(a):
                            src = t(w-d)
                            dest = t(w+d)
                            valid = True

#                     Target cell contains an operator
                    if valid and self.is_op(self.grid[dest]):
                        s = self.grid[src]
#                         if False == True and type(s) is int or s.isnumeric():
                        if False:
                            if new[dest] == '&':
                                new[dest] = 1#int(s)
                            else:
                                new[dest] = initial[new[dest]]
                        
                        new[dest].inputs.append(self.grid[src])

                    if valid and False:
                        func = self.grid[dest]
    #                     Check that the operator should be applied
                        if self.grid[dest] in self.ops and type(new[dest]) is int and self.grid[src] != '_':
                            src_val = int(self.grid[src])
                            if func == '+':
                                new[dest] += src_val
                            elif func == '*':
                                new[dest] *= src_val
                            elif func == '$':
                                new[dest] += src_val
                            elif func == '&':
                                if new[dest] == 1:
                                    new[dest] = src_val
                                else:
                                    new[dest] **= src_val
                                    
            for x, y in np.ndindex(new.shape):
                pos = (x, y)
                c = new[pos]
                if self.is_op(c):
                    c.evaluate()

#             Copy the new frame
            self.grid = new.copy()

    def display(self, empty=' '):
        printout = self.grid.copy()
        for x, y in np.ndindex(printout.shape):
            p = str(printout[x][y])
            if self.grid.shape[1] > len(p) > 1:
                printout[x][y:y+len(p)+1] = list(p+']')
        printout = '\n'.join(''.join(str(s) for s in r) for r in printout)
        print(printout.replace('_', empty),'\n')
                    
L = Locus(sample)
L.execute(i=3)
L.display()

                                                                       
                                       2>2                             
 5 6-6-8                                 v          3>3                
 |/    |     3-7-4                     9>16]>16]      v                
 60]   2                                 ^          6>11]              
 |           7-7-7                       5            v                
 60]           | |                                  7>18]>18]          
               7-14]       3>7>9>9>9                  v     v          
                             ^ ^ ^ ^                  18]   18]        
                             4 2                                       
   2-2                                                                 
     v                                                                 
   1>6<3                                        3>3                    
     v                                            v             

In [21]:
(5, 3) + (2, 9)

(5, 3, 2, 9)